Implementation of GPT-2

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-18 07:56:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    in 0,07s   

2023-01-18 07:56:53 (14,3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [43]:
import torch

In [44]:
with open("input.txt", "r", encoding="UTF-8") as f:
    text = f.read()
print(len(text))
print(text[:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [45]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [46]:
# create encoder and decoder
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

# testing
example = "hii there"
print(encode(example))
print(decode(encode(example)))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [47]:
data = torch.tensor(encode(text), dtype=torch.long)

In [48]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [49]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - batch_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch("train")
print(xb.shape, yb.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [56]:
eval_iter = 500

In [57]:
@torch.no_grad()
def estimate_loss():
    out = {}
    bm.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iter)
        for k in range(eval_iter):
            X, Y = get_batch(split)
            logits, loss = bm(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    bm.train()
    return out

In [58]:
# Implementing bigram to have a benchmark

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C) expected: (4, 8, 65)

        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

bm = BigramLanguageModel(vocab_size)
logits, loss = bm(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(bm.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.6798, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [59]:
optimizer = torch.optim.AdamW(bm.parameters(), lr=1e-3)

In [60]:
batch_size = 32
total_steps = 10000
for step in range(total_steps):

    if step % eval_iter == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch("train")

    logits, loss = bm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step <built-in function iter>: train loss 4.7280, val loss 4.7195
step <built-in function iter>: train loss 4.1775, val loss 4.1726
step <built-in function iter>: train loss 3.7320, val loss 3.7343
step <built-in function iter>: train loss 3.3797, val loss 3.3890
step <built-in function iter>: train loss 3.1213, val loss 3.1250
step <built-in function iter>: train loss 2.9358, val loss 2.9423
step <built-in function iter>: train loss 2.7985, val loss 2.8043
step <built-in function iter>: train loss 2.7064, val loss 2.7148
step <built-in function iter>: train loss 2.6409, val loss 2.6473
step <built-in function iter>: train loss 2.5888, val loss 2.5970
step <built-in function iter>: train loss 2.5668, val loss 2.5759
step <built-in function iter>: train loss 2.5381, val loss 2.5464
step <built-in function iter>: train loss 2.5162, val loss 2.5329
step <built-in function iter>: train loss 2.5030, val loss 2.5220
step <built-in function iter>: train loss 2.4924, val loss 2.5154
step <buil

In [41]:
print(decode(bm.generate(idx, max_new_tokens=300)[0].tolist()))



TYoreriven, porthar' wially'donkin, ENO, trd onkn t o y oceneloo wieredices moo tomy d: fopr, HAnd hacorusey tind pist manow, llly anit odosththo pong whanof stherfitou tee.
Wighedave foutuld eay t ond sthtouer mpo;
G in f bonod can pe harintarthil I Jund

YBe, qure ben bouce'dg, urthougef PEDus ro
